# NeuroFetal AI - Enhanced Fusion ResNet Training

**Version 2.0** - Advanced Enhancements:
- 3-Input Architecture (FHR + Tabular + CSP)
- Squeeze-and-Excitation (SE) Blocks
- Multi-Head Self-Attention
- Focal Loss for Class Imbalance

### Instructions:
1. **Runtime → Change runtime type → GPU (T4)**
2. Run cells in order
3. Use **🔄 Update Repo** cell anytime to pull latest changes

## Step 0: GitHub Authentication

In [ ]:
from getpass import getpass
import os

GITHUB_REPO = "Krishna200608/NeuroFetal-AI"
GITHUB_TOKEN = getpass("GitHub Personal Access Token: ")

os.environ['GITHUB_TOKEN'] = GITHUB_TOKEN
os.environ['GITHUB_REPO'] = GITHUB_REPO
print("✅ Token saved to session.")

## Step 1: Clone Repository

In [ ]:
import os
import shutil

# Always start from /content
os.chdir("/content")

GITHUB_REPO = os.environ.get('GITHUB_REPO', 'Krishna200608/NeuroFetal-AI')
GITHUB_TOKEN = os.environ.get('GITHUB_TOKEN', '')

# Clean clone for fresh start
if os.path.exists("NeuroFetal-AI"):
    shutil.rmtree("NeuroFetal-AI")
    print("🗑️ Removed old clone.")

print("📥 Cloning repository...")
if GITHUB_TOKEN:
    !git clone https://{GITHUB_TOKEN}@github.com/{GITHUB_REPO}.git
else:
    !git clone https://github.com/{GITHUB_REPO}.git

# Verify
dataset_path = "/content/NeuroFetal-AI/Datasets/ctu_uhb_data"
if os.path.exists(dataset_path):
    count = len([f for f in os.listdir(dataset_path) if f.endswith('.dat')])
    print(f"\n✅ SUCCESS: Found {count} dataset records!")
else:
    print(f"\n❌ ERROR: Dataset folder not found!")

## 🔄 Update Repo (Run anytime to pull latest changes)
**Run this cell whenever you make changes on GitHub or encounter errors.**

In [ ]:
import os
os.chdir("/content/NeuroFetal-AI")
!git pull origin main
print("\n✅ Repository updated! Re-run training cells below.")

## Step 2: Setup Paths

In [ ]:
import os
import sys

PROJECT_ROOT = "/content/NeuroFetal-AI"
CODE_DIR = os.path.join(PROJECT_ROOT, "Code")
SCRIPTS_DIR = os.path.join(CODE_DIR, "scripts")
UTILS_DIR = os.path.join(CODE_DIR, "utils")

os.chdir(SCRIPTS_DIR)
sys.path.insert(0, SCRIPTS_DIR)
sys.path.insert(0, UTILS_DIR)
sys.path.insert(0, CODE_DIR)

print(f"📂 Working dir: {os.getcwd()}")

## Step 3: Install Dependencies

In [ ]:
!pip install -q wfdb shap scipy

In [ ]:
import tensorflow as tf
print(f"TensorFlow: {tf.__version__}")
print(f"GPU: {tf.config.list_physical_devices('GPU')}")

## Step 4: Data Ingestion

In [ ]:
import os
processed_dir = "/content/NeuroFetal-AI/Datasets/processed"

if os.path.exists(os.path.join(processed_dir, "X_fhr.npy")):
    print("✅ Processed data exists. Skipping ingestion.")
else:
    print("⏳ Running data ingestion...")
    !python data_ingestion.py

In [ ]:
import numpy as np

X_fhr = np.load(os.path.join(processed_dir, "X_fhr.npy"))
X_tabular = np.load(os.path.join(processed_dir, "X_tabular.npy"))
y = np.load(os.path.join(processed_dir, "y.npy"))

print(f"FHR: {X_fhr.shape}, Tabular: {X_tabular.shape}, Labels: {y.shape}")
print(f"Class Balance: {np.mean(y):.2%} positive")

## Step 5: Training

In [ ]:
!python train.py

## Step 6: Explainability (XAI)

In [ ]:
!python xai.py

## Step 7: Push Results to GitHub

In [ ]:
!git config --global user.email "krishnabhujel2006@gmail.com"
!git config --global user.name "Krishna Bhujel"

In [ ]:
import os
os.chdir("/content/NeuroFetal-AI")

!git add Code/models/*.keras Reports/training_logs/*.json Code/figures/*.png 2>/dev/null || true
!git status
!git commit -m "Add trained model from Colab" || echo "Nothing to commit"
!git push origin main

## Results

In [ ]:
import json, glob, os

log_dir = "/content/NeuroFetal-AI/Reports/training_logs"
logs = sorted(glob.glob(os.path.join(log_dir, "training_log_*.json")))

if logs:
    with open(logs[-1]) as f:
        r = json.load(f)
    print(f"📊 Mean AUC: {r['summary']['mean_auc']:.4f} ± {r['summary']['std_auc']:.4f}")
else:
    print("No training logs found.")